In [9]:
import pandas as pd
import sqlite3

In [10]:
db_con = sqlite3.connect('../checking-logs.sqlite.sqlite')
query = "PRAGMA table_info(checker);"
pd.io.sql.read_sql(query, db_con)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


In [12]:
query = "SELECT * FROM test LIMIT 10"
pd.io.sql.read_sql(query, db_con)

DatabaseError: Execution failed on sql 'SELECT * FROM test LIMIT 5': no such table: test

In [1]:
query = "SELECT test.uid, MIN(CAST((julianday(test.first_commit_ts) - \
      julianday(datetime(deadlines.deadlines, 'unixepoch'))) \
     * 24 AS integer)) \
AS dif_time FROM test \
JOIN deadlines ON test.labname=deadlines.labs \
WHERE labname<>'project1'"
df_min = pd.io.sql.read_sql(query, db_con)
df_min

NameError: name 'pd' is not defined

In [15]:
query = "SELECT test.uid, MAX(CAST((julianday(test.first_commit_ts) - \
      julianday(datetime(deadlines.deadlines, 'unixepoch'))) \
     * 24 AS integer)) \
AS dif_time FROM test \
JOIN deadlines ON test.labname=deadlines.labs \
WHERE labname<>'project1'"
df_max = pd.io.sql.read_sql(query, db_con)
df_max

,uid,dif_time
0,user_25,-2


In [16]:
query = "SELECT AVG(CAST((julianday(test.first_commit_ts) - \
      julianday(datetime(deadlines.deadlines, 'unixepoch'))) \
     * 24 AS integer)) \
AS dif_time FROM test \
JOIN deadlines ON test.labname=deadlines.labs \
WHERE labname<>'project1'"
df_avg = pd.io.sql.read_sql(query, db_con)
df_avg

,dif_time
0,-89.125


In [17]:
query = '''SELECT test.uid as uid,
AVG(CAST((julianday(test.first_commit_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch')))* 24 AS integer))
AS avg_diff, views.av AS pageviews
FROM test
JOIN deadlines ON test.labname=deadlines.labs
JOIN (SELECT uid, COUNT(uid) AS av from pageviews GROUP By uid) AS views ON test.uid=views.uid
WHERE labname<>'project1'
GROUP BY test.uid
'''
views_diff = pd.io.sql.read_sql(query, db_con)
views_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,28
1,user_10,-74.800000,89
2,user_14,-159.000000,143
3,user_17,-61.600000,47
4,user_18,-5.666667,3
5,user_19,-98.750000,16
6,user_21,-95.500000,10
7,user_25,-92.600000,179
8,user_28,-86.400000,149
9,user_3,-105.400000,317


In [18]:
views_diff.corr()

,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


In [19]:
db_con.close()